In [185]:
class IntegerValue:

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance, None)

In [186]:
import weakref

In [187]:
class IntegerValue:

    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance, None)

In [188]:
class Point:
    x= IntegerValue()

In [189]:
p= Point()

In [190]:
print(hex(id(p)))

0x23592869db0


In [191]:
p.x = 100.6

In [192]:
p.x

100

In [193]:
Point.x.values.keyrefs()

[<weakref at 0x000002359439B4C0; to 'Point' at 0x0000023592869DB0>]

In [194]:
del p

In [195]:
Point.x.values.keyrefs()

[]

In [196]:
class IntegerValue: # for non hashable classes

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(id(instance))

In [197]:
class Point:
    x= IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x


In [198]:
p = Point(10.5)

In [199]:
p.x

10

In [200]:
p.x = 20.1

In [201]:
p.x

20

In [202]:
id(p), Point.x.values

(2429114812640, {2429114812640: 20})

In [203]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [204]:
p_id = id(p)

In [205]:
ref_count(p_id)

1

In [206]:
del p

In [207]:
ref_count(p_id)

0

In [208]:
Point.x.values

{2429114812640: 20}

In [209]:
p =Point(1.1)

In [210]:
weak_p = weakref.ref(p)

In [211]:
weak_p

<weakref at 0x00000235944C2110; to 'Point' at 0x0000023592847550>

In [212]:
ref_count(id(p))

1

In [213]:
del p

In [214]:
weak_p # somehow inform that -> is dead

<weakref at 0x00000235944C2110; dead>

In [215]:
# We can specify func to be called after becoming dead

In [216]:
def obj_destroyed(obj):
    print(f"{obj}, is being destroyed")

In [217]:
p = Point(1.111212)

In [218]:
w = weakref.ref(p, obj_destroyed)

In [219]:
del p

<weakref at 0x00000235944D8270; dead>, is being destroyed


In [220]:
w

<weakref at 0x00000235944D8270; dead>

In [221]:
class IntegerValue: # for non hashable classes

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        value_tuple = self.values[(id(instance))]
        return value_tuple[1]

    def _remove_object(self, weak_ref):
        print(f'removin deaad entry from {weak_ref}')

In [222]:
class Point:
    x = IntegerValue()

In [223]:
p1 = Point()
p2 = Point()

removin deaad entry from <weakref at 0x00000235944C2340; dead>


In [224]:
p1.x, p2.x = 10.2, 10001.6

In [225]:

p1.x, p2.x

(10, 10001)

In [226]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [227]:
del p1

removin deaad entry from <weakref at 0x00000235944C1580; dead>


In [228]:
class IntegerValue: # for non hashable classes

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        value_tuple = self.values[(id(instance))]
        return value_tuple[1]

    def _remove_object(self, weak_ref):
        '''
        reverse_lookup = [ key for key, value in self.values.items()
                            if value[0] is weak_ref ] 
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]
        '''
        for key, value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break

In [229]:
class Point:
    x = IntegerValue()

In [230]:
p = Point()

In [231]:
p.x = 236.32

In [232]:
p.x

236

In [233]:
Point.x.values

{2429114813840: (<weakref at 0x000002359426B880; to 'Point' at 0x0000023592868D90>,
  236)}

In [234]:
del p

In [235]:
Point.x.values

{}

In [236]:
class Person:
    pass

In [237]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [238]:
hasattr(Person.__weakref__, '__set__')

True

In [239]:
p = Person()

In [240]:
w = weakref.ref(p)

In [241]:
class Person:
    __slots__ = 'name',

In [242]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [243]:
p =Person()

In [244]:
hasattr(p, '__weakref__')

False

In [250]:
w = weakref.ref(p)

In [251]:
class Person:
    __slots__ = 'name', '__weakref__'

In [252]:
p = Person()

In [253]:
weakref.ref(p)

<weakref at 0x0000023593986E30; to 'Person' at 0x0000023592868490>

In [254]:
class ValidString:
    def __init__(self, min_lenght=0, max_lenght=255):
        self.data = {}
        self._min_lenght = min_lenght
        self._max_lenght = max_lenght

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError ('Value must be a str')

        if len(value) < self._min_lenght:
            raise ValueError(
                f'Value should be at least {self._min_lenght} characters'
            )
        if len(value) > self._max_lenght:
            raise ValueError(
                f'Value cannot exeed {self._min_lenght} characters'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
    
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [ key for key, value in self.data.items()
                            if value[0] is weak_ref ] 
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [255]:
class Person:
    __slots__ = "__weakref__",

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return(
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name  == other.last_name
        )

class BankAccount:
    __slots__ = "__weakref__",

    account_number = ValidString(5, 255)

    def __eq__(self, other) -> bool:
        return isinstance(other, BankAccount) and self.account_number == other.account_number
    



In [256]:
p1 = Person()

In [257]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters


In [258]:
p2 = Person()

removin deaad entry from <weakref at 0x0000023594489FD0; dead>


In [259]:
p1.first_name, p1.last_name = 'Ja', 'Va'
p2.first_name, p2.last_name = 'Py', 'Thon'

In [260]:
b1, b2 = BankAccount(), BankAccount()

In [261]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [262]:
p1.first_name, p1.last_name

('Ja', 'Va')

In [263]:
p2.first_name, p2.last_name

('Py', 'Thon')

In [264]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [265]:
Person.first_name.data

{2429114812976: (<weakref at 0x000002359438ECA0; to 'Person' at 0x0000023592868A30>,
  'Ja'),
 2429114820608: (<weakref at 0x000002359451C310; to 'Person' at 0x000002359286A800>,
  'Py')}

In [266]:
BankAccount.account_number.data

{2429114816480: (<weakref at 0x00000235944FD9E0; to 'BankAccount' at 0x00000235928697E0>,
  'Savings'),
 2429114823632: (<weakref at 0x00000235944FDCB0; to 'BankAccount' at 0x000002359286B3D0>,
  'Checking')}

In [267]:
del p1
del p2
del b1
del b2

In [268]:
Person.first_name.data # something wrong before there was a one weak ref 

{}

In [123]:
ref_count(2429114816432) # there was 2:   an exeption above make a ref should use try

2

In [269]:
BankAccount.account_number.data

{}